# Replication: Agosta *et al*, 2013

## Introduction

This notebook attempts to replicate the following paper with the [PPMI](http://ppmi-info.org) dataset:

<div class="alert alert-block alert-success">
Agosta, Federica, et al. <a href="https://onlinelibrary.wiley.com/doi/abs/10.1002/hbm.22101?casa_token=vStnHL7zZ9MAAAAA:Ua2XOtP-ZFkkN-ebLPWi-84jCctqFpC8I89zyE3Ia9Lvk8bJWl7wj7-15xciyX50gzG05TlCvv6DFk0">
The topography of brain damage at different stages of Parkinson's disease.</a>, Human brain mapping 34.11 (2013): 2798-2807.
</div>

<img src='images/table.png' width=800/>

In [1]:
# H&Y during OFF time
# Exclusion criteria: Patients were excluded if they had:
# (a) parkin, leucine-rich repeat kinase 2 (LRRK2), and glu-
# cocerebrosidase (GBA) gene mutations;  OK
# (b) cerebrovascular
# disorders, traumatic brain injury history, or intracranial
# mass; NOT FOUND IN PPMI
#
# (c) other major neurological and medical diseases;
# (d) dementia: OK

# DARTEL:  8-mm full width
# at half maximum (FWHM) Gaussian filter

## Initial setup

In [2]:
import livingpark_utils

utils = livingpark_utils.LivingParkUtils("agosta-etal")
utils.prologue()
random_seed = 1

Installing notebook dependencies (see log in install.log)... 
This notebook was run on 2022-07-19 07:27:34.951400


## PPMI cohort preparation

### Study data download

In [3]:
required_files = [
    "iu_genetic_consensus_20220310.csv",
    "Cognitive_Categorization.csv",
    "Primary_Clinical_Diagnosis.csv",
]

utils.install_ppmi_study_files(required_files)

The following files are now available: ['iu_genetic_consensus_20220310.csv', 'Cognitive_Categorization.csv', 'Primary_Clinical_Diagnosis.csv']


In [32]:
# H&Y scores

# TODO: move this to livingpark_utils
import os.path as op

file_path = op.join(utils.study_files_dir, "MDS_UPDRS_Part_III_clean.csv")
if not op.exists(file_path):
    !(cd {utils.study_files_dir} && python -m wget "https://raw.githubusercontent.com/LivingPark-MRI/ppmi-treatment-and-on-off-status/main/PPMI medication and ON-OFF status.ipynb")  # use requests to improve portability
    npath = op.join(utils.study_files_dir, "PPMI medication and ON-OFF status.ipynb")
    %run "{npath}"
print(f"File {file_path} is now available")

# TODO: move this to livingpark_utils
import os.path as op

file_path = op.join(utils.study_files_dir, "MRI_info.csv")
if not op.exists(file_path):
    !(cd {utils.study_files_dir} && python -m wget "https://raw.githubusercontent.com/LivingPark-MRI/ppmi-MRI-metadata/main/MRI metadata.ipynb")  # use requests to improve portability
    npath = op.join(utils.study_files_dir, "MRI metadata.ipynb")
    %run "{npath}"
print(f"File {file_path} is now available")

File inputs/study_files/MDS_UPDRS_Part_III_clean.csv is now available
File inputs/study_files/MRI_info.csv is now available


### Inclusion criteria

We obtain the following group sizes:

<!-- and a cutoff score of 6 to identify RBD subjects among PD subjects, consistently with the results presented in [[2]](https://www.sciencedirect.com# /science/article/pii/S138994571100164X). -->

In [36]:
import pandas as pd

# UPDRS3
updrs3 = pd.read_csv(op.join(utils.study_files_dir, "MDS_UPDRS_Part_III_clean.csv"))[
    ["PATNO", "EVENT_ID", "PDSTATE", "NHY"]
]
# Genetics
genetics = pd.read_csv(
    op.join(utils.study_files_dir, "iu_genetic_consensus_20220310.csv")
)[["PATNO", "GBA_PATHVAR", "LRRK2_PATHVAR", "NOTES"]]
# Cognitive Categorization
cog_cat = pd.read_csv(op.join(utils.study_files_dir, "Cognitive_Categorization.csv"))[
    ["PATNO", "EVENT_ID", "COGSTATE"]
]
# Diagnosis
diag = pd.read_csv(op.join(utils.study_files_dir, "Primary_Clinical_Diagnosis.csv"))[
    ["PATNO", "EVENT_ID", "PRIMDIAG", "OTHNEURO"]
]
# MRI
mri = pd.read_csv(op.join(utils.study_files_dir, 'MRI_info.csv'))[["Subject ID", "Visit code", "Description"]]
mri.rename(columns={"Subject ID": "PATNO", "Visit code": "EVENT_ID"}, inplace=True)

In [84]:
# Remove genetics data that should be excluded according to PPMI doc
genetics_clean = genetics[
    ~(
        (genetics["NOTES"].notna())
        & (
            genetics["NOTES"].str.contains("exclude GBA")
            | genetics["NOTES"].str.contains("exclude LRRK2")
        )
    )
]

# Inclusion / exclusion criteria
subjects = (
    updrs3[
        (updrs3["PDSTATE"] == "OFF") & (updrs3["NHY"].notna()) & (updrs3["NHY"]!="UR")
    ]  # H&Y score is available and was obtained during OFF time
    .merge(
        genetics_clean[  # No pathogenic GBA or LRRK2 variant present (Note: on-going email thread with Madeleine)
            (genetics_clean["GBA_PATHVAR"] == 0)
            & (genetics_clean["LRRK2_PATHVAR"] == 0)
        ],
        how="inner",
        on="PATNO",
    )
    .merge(
        cog_cat[cog_cat["COGSTATE"] != 3], how="inner", on=["PATNO", "EVENT_ID"]
    )  # No dementia
    .merge(
        diag[
            (diag["PRIMDIAG"].isin([1, 17])) & (diag["OTHNEURO"].isna())
        ],  # Primary diagnosis is Idiopathic PD or healthy control
        how="inner",
        on=["PATNO", "EVENT_ID"],
    )
    .merge(
        mri,
        how='inner',
        on=['PATNO', 'EVENT_ID']
    )
)

pds = subjects[subjects["PRIMDIAG"] == 1]
controls = subjects[(subjects["PRIMDIAG"] == 17) & (subjects['NHY']=='0')]  # Exclude controls with H&Y > 0
subjects = pd.concat([pds, controls])


print(f"Number of controls: {len(pd.unique(controls['PATNO']))}")
print(f"Number of PD subjects: {len(pd.unique(pds['PATNO']))}")

print(f"Number of PD subject visits by H&Y score:")
pds.groupby(['NHY']).count()[['PATNO']]

Number of controls: 47
Number of PD subjects: 128
Number of PD subject visits by H&Y score:


,PATNO
NHY,
1,68
2,193
3,17
4,3


### Cohort matching

In [93]:
sample_size = {
0: 42,
1: 17,
2: 46,
3: 14,
4: 12
}

samples = {}

subjects_ = subjects.copy()  # copy subjects DF to leave it intact
for x in sorted(sample_size, reverse=True): # sampling patients in descending H&Y score order as there are less patients with high H&Y scores
    print(f"Selecting {sample_size[x]} visits of unique patients with H&Y={x}")
    try:
        # Sample visits with H&Y=x and make sure they belong to different patients
        hy = subjects_[subjects_['NHY']==str(x)].groupby(['PATNO']).sample(1, random_state=random_seed)
        samples[x] = hy.sample(sample_size[x], random_state=random_seed)
    except Exception as e:
        if 'Cannot take a larger sample than population' in str(e):
            print(f'Warning: found only {len(hy)} visits of unique patients with H&Y={x} while {sample_size[x]} were required')
            samples[x] = hy
        else:
            raise(e)
    # Remove sampled patients from subjects_
    subjects_ = subjects_[~(subjects_['PATNO'].isin(samples[x]['PATNO']))]


Selecting 12 visits of unique patients with H&Y=4
Selecting 14 visits of unique patients with H&Y=3
Selecting 46 visits of unique patients with H&Y=2
Selecting 17 visits of unique patients with H&Y=1
Selecting 42 visits of unique patients with H&Y=0


In [94]:
cohort = pd.concat([samples[x] for x in samples])
cohort.groupby('NHY').count()

,PATNO,EVENT_ID,PDSTATE,GBA_PATHVAR,LRRK2_PATHVAR,NOTES,COGSTATE,PRIMDIAG,OTHNEURO,Description
NHY,,,,,,,,,,
0,42,42,42,42,42,0,42,42,0,42
1,17,17,17,17,17,2,17,17,0,17
2,46,46,46,46,46,1,46,46,0,46
3,10,10,10,10,10,0,10,10,0,10
4,2,2,2,2,2,0,2,2,0,2
